In [88]:
import pandas as pd
from dateutil.parser import parse


In [123]:
supplier_feed = pd.read_csv('../data/supplier_feed.csv')
print("CSV file read successfully.")

CSV file read successfully.


In [124]:
supplier_feed.head()

,part_id,stock_level,cost_price,entry_date
0,SP-236,Low Stock,287.32,2024-06-29
1,SP-325,248,388.87,"Mar 17, 2025"
2,SP-179,82,109,2025-04-14
3,SP-332,171,419.6,2025-05-10
4,SP-268,72,279.69,2024-04-08


In [125]:
supplier_feed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   part_id      30000 non-null  object
 1   stock_level  26064 non-null  object
 2   cost_price   28463 non-null  object
 3   entry_date   30000 non-null  object
dtypes: object(4)
memory usage: 937.6+ KB


In [126]:
supplier_feed.describe()

,part_id,stock_level,cost_price,entry_date
count,30000,26064,28463,30000
unique,400,501,21333,2305
top,SP-468,LOW,358,"Jan 04, 2025"
freq,98,1012,19,36


In [127]:
def is_integer(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [128]:
non_int_stock_level_mask = ~supplier_feed['stock_level'].apply(is_integer)
non_int_stock_level_values = supplier_feed.loc[non_int_stock_level_mask, 'stock_level'].unique()
print(non_int_stock_level_values)



['Low Stock' 'UNAVAILABLE' 'low stock' nan 'Out of Stock' 'LOW']


In [129]:
supplier_feed.fillna({'stock_level': '-1'}, inplace=True)

In [151]:
supplier_feed["stock_level_clean"] = supplier_feed["stock_level"]

supplier_feed["stock_level_clean"]

0          Low Stock
1                248
2                 82
3                171
4                 72
            ...     
29995            385
29996             -1
29997             -1
29998    UNAVAILABLE
29999            350
Name: stock_level_clean, Length: 30000, dtype: object

In [154]:
binary_map = {
    "low stock": 1,
    "low": 1,
    "unavailable": 0,
    "out of stock": 0
}

def clean_stock(s):
    try:
        return int(s)
    except ValueError:
        # not a number, treat as text
        s_norm = str(s).strip().lower()
        return binary_map.get(s_norm, None)

supplier_feed["stock_level_clean"] = supplier_feed["stock_level_clean"].apply(clean_stock)


In [155]:
supplier_feed["stock_level_clean"]

0          1
1        248
2         82
3        171
4         72
        ... 
29995    385
29996     -1
29997     -1
29998      0
29999    350
Name: stock_level_clean, Length: 30000, dtype: int64

In [95]:
non_int_cost_price_mask = ~supplier_feed['cost_price'].apply(is_float)
non_int_cost_price_values = supplier_feed.loc[non_int_cost_price_mask, 'cost_price'].unique()
print(non_int_cost_price_values)

['$203.60' '$45.49' '$285.43' ... '$21.53' '$146.14' '$288.32']


In [96]:
def remove_dollar_sign(value):
    return str(value).replace('$', '')

supplier_feed['cost_price_filter'] = supplier_feed['cost_price'].apply(remove_dollar_sign)

In [97]:
supplier_feed['cost_price_filter'].isna().sum()

np.int64(0)

In [98]:
print(supplier_feed["entry_date"])

0                 2024-06-29
1               Mar 17, 2025
2                 2025-04-14
3                 2025-05-10
4                 2024-04-08
                ...         
29995             2025-04-22
29996             2024-03-17
29997    2024-09-18T00:00:00
29998               04/26/25
29999               06/29/25
Name: entry_date, Length: 30000, dtype: object


In [99]:
EXPLICIT_FORMATS = [
    "%Y-%m-%d", "%Y/%m/%d",
    "%m/%d/%Y", "%m/%d/%y",
    "%d/%m/%Y", "%d/%m/%y",
    "%b %d, %Y",  # Mar 17, 2025
    "%B %d, %Y",  # March 17, 2025
    "%Y-%m-%dT%H:%M:%S", "%Y-%m-%d %H:%M:%S",
    "%Y-%m-%dT%H:%M:%S.%f", "%Y-%m-%d %H:%M:%S.%f",
]

def _clean(s):
    if pd.isna(s): return s
    s = str(s).replace("\u00A0", " ").strip()
    s = s.strip('\'"“”‘’`')  # strip common quotes
    return s

def parse_any(x):
    s = _clean(x)
    if pd.isna(s) or s == "": return pd.NaT
    # try explicit formats first
    for fmt in EXPLICIT_FORMATS:
        try:
            return pd.to_datetime(s, format=fmt)
        except Exception:
            pass
    # let pandas try generic parse
    try:
        return pd.to_datetime(s, errors="raise")
    except Exception:
        pass
    # final fallback: dateutil (very tolerant)
    try:
        return parse(s, dayfirst=False)
    except Exception:
        return pd.NaT

supplier_feed["entry_date_dt"]  = supplier_feed["entry_date"].apply(parse_any)

In [100]:
supplier_feed.head()

,part_id,stock_level,cost_price,entry_date,cost_price_filter,entry_date_dt
0,SP-236,Low Stock,287.32,2024-06-29,287.32,2024-06-29
1,SP-325,248,388.87,"Mar 17, 2025",388.87,2025-03-17
2,SP-179,82,109,2025-04-14,109,2025-04-14
3,SP-332,171,419.6,2025-05-10,419.6,2025-05-10
4,SP-268,72,279.69,2024-04-08,279.69,2024-04-08
